In [ ]:
function initialize_BP()
    
    if BP_INIT_VERSION == "v1"
        # Ajout d'une colonne artificielle pour initialer l'algorithme de branchement
        artifcolumn = Dict{String,Any}() 
        artifcolumn["cost"] = minimum(WEIGHTS)[2]
        artifcolumn["vertices"] = ones(Int,nv(GRAPH))
        artifcolumn["oneedges"] = []
        push!(column_pool, artifcolumn)
        global KEEP_INIT_COLUMNS = 1 # Nombre de colonnes initiales qui seront conservées lors d'un branchement
        
    elseif BP_INIT_VERSION == "v2"
        println("BP_INIT_VERSION == v2")
        global KEEP_INIT_COLUMNS = "autre chose ? peut-être 0"
    end
end

In [ ]:
function calculate_branching(x)
    arc_src,arc_dst = 0,0
    
    for (i,j) in EDGES
        if modf(x[(i,j)])[1] > ϵ  # Recherche d'une variable fractionnaire
            arc_src,arc_dst = i,j
            break
        end
    end
    return (arc_src,arc_dst)
end

In [ ]:
function solve_BP()
    global column_pool = Array{Dict{String,Any},1}()
    # 'column_pool' est l'ensemble de toutes les colonnes ajoutées au problème maître restreint
    # Le premier indice correspond à l'indice de colonne c.
    # Dans la colonne column_pool[c] se trouve un dictionnaire contenant deux éléments :
    #  - "cost"     : cout de la colonne
    #  - "vertices" : liste contenant les valeurs des variables y_i, où i est un couple donneur/receveur
    #  - "oneedges" : liste des arcs du graphe qui sont conservés dans ce cycle (une colonne représente un cycle)
    
    # Initialisation du Branch and Price (colonnes artificielles, utilisation d'une heuristic)
    initialize_BP()

    # Bornes inférieures et supérieures du problème de départ, mise pour commencer à -infini et +infini
    global LB = -Inf
    global UB = Inf

    global Queue = Vector{Int}() # Contient les indices des noeuds qui doivent être encore être traités
    global tree = Vector{TreeNode}() # Arbre de branchement (sous forme de liste) qui contiendra des objets TreeNode
    
    # L'arbre de branch and price est initialisé avec un noeud racine numéro 1
    #   Le noeud racine est son propre parent, il n'a pas encore d'enfants, à pour borne supérieure +infini,
    #   et n'a aucune contrainte de branchement
    push!(tree,TreeNode(0,[],Inf,[],[]))   
    push!(Queue,1)  # ajout de l'indice 1 qui correspond au noeud racine, et qu'il reste à traité pour l'instant
    
    
    # Tant qu'il reste des à traiter, on branche
    while length(Queue) > 0
        println("Liste des noeuds qu'il reste à traiter : ", Queue)

        current = Queue[end] # noeud que l'on traite dans cette boucle

        x = Process_Node(current) # Résoud le noeud courant en calculant sa borne duale (supérieure ici)
                                  # et retourne la liste des variables x_ij pour tous les arcs
                                  # ou bien une liste vide [] si le noeud est irréalisable
      
        # Si le noeud courant est réalisable et que sa borne supérieure est prometteuse alors on branche
        if x != [] && tree[current].ub >= LB
            
            # La solution est-elle entière ?
            (i,j) = calculate_branching(x) # Si oui, 'calculate_branching(x)' renvoie (0,0)
                                           # sinon, la fonction renvoie un arc x_ij qui à une valeur fractionnaire
            
            # Si la solution n'est pas entière, alors on branche sur l'arc (i,j) trouvé
            if (i,j) != (0,0)
                println("Deux noeuds sont créés en branchant sur x_($i,$j)")
                push!(tree, TreeNode(current,[],tree[current].ub,vcat((i,j),tree[current].setzero),tree[current].setone))
                push!(tree[current].children, length(tree))
                push!(Queue, length(tree))
                push!(tree, TreeNode(current,[],tree[current].ub,tree[current].setzero,vcat((i,j),tree[current].setone)))
                push!(tree[current].children, length(tree))
                push!(Queue, length(tree))
                
            # sinon, si la solution est entière, alors on a résolu cette branche de l'arbre
            else
                println(" \e[93m|\e[92m  La solution relâchée optimale était entière")
                println(" \e[93m|\e[92m  Valeur réalisable $(tree[current].ub) trouvée\e[00m")
            end
        
        # sinon, si le noeud est irréalisable ou non-prometteur, alors on l'élague
        else
            println(" \e[93m|\e[31m  Ce noeud est élagué car irréalisable ou car sa borne est non-prometteuse \e[00m")
        end
        
        # Calcul de la borne supérieure globale comme le maximum des bornes supérieures de tous les noeuds actifs
        global UB = tree[current].ub
        for i in Queue
            if tree[i].ub >= UB
                global UB = tree[i].ub
            end
        end
        
        # Affichage des bornes inf et sup globales à cet instant
        println("\e[94mLB=$LB, UB=$UB \e[00m")
        
        # on retire le noeud traité de la liste 'Queue'
        deleteat!(Queue,findfirst(x -> x == current, Queue))
        # on retire tous les noeuds dont la borne supérieure n'est pas prometteuse
        deleteat!(Queue,unique(nodestobedeleted))
        
        # Si le gap d'optimalité est assez faible alors on s'arrête
        if abs(2*(UB-LB)/(UB+LB))<=ϵ
            break
        end
    end
end